# Requirements to Run

Instructions to run are in the read me

The main thing to run is `sim_battle.py`, if you want to run the data_parsing and model stuff the libraries used are below

## Libraries

So far we have used the open source Pokémon Showdown code and the poke-env library.

https://github.com/hsahovic/poke-env/tree/51a025929f6c0164ce3e263af0cab286f79b2f5c

https://github.com/smogon/pokemon-showdown

https://github.com/Robin99999/Pokemon_Showdown_AI

 The poke-env library is needed because the Pokémon Showdown server is written in JavaScript, and the poke-env library allows us to simulate Pokémon Showdown while in Python.

To parse the data we used requests, json, os, numpy, copy, re, random, math

The ml model uses tensorflow,numpy, pandas, sci-kit learn

## What the code does

-------NOTE: our project is not finished even though we have both spent around 8-hours a day since last Wednesday working on this project, the project is definitely out of scope for this class and we will be contiunuing this project as a passion project. I hope that given the scope of our project its understandable why we dont have a fully optimized and functional minimax player.

------data_fixing.ipynb is the code we used to parse the data from the replay files. The replays are record from the pokemon showdown website where copy and pasted 50 replays at a time into the replay_links folder.

The data_fixing.ipynb shows the data structure we used to record each game_state during the pokemon showdown game (open the notebook to see how much data we need to keep track of for each turn of the game). 

Also checkout what we are parsing in the all_un_parsed_replays folder where it shows the json we recieve from the pokemon showdown server. At the end this code exports everything to 12-11-game-states.json. 

We only record games that have a rating higher than 1250 so the players know the game, and we are only recording 30% of the turns per game so that we do not overfit. We populate the estimated win rate ourselves for this data based on who won. 

We use a sigmoid function so that as the game goes on we are more confident in winning/losing and less confident at the beginning of the game. 

There are more details in the code I won't go over here but all the data we record of every game state is important to keep track of in the game of pokemon. There are still things our data_structure is missing like keeping track of "yawn", "encore", 'disable' but there are so many niche moves in pokemon that if we tried to account for all of them it would take so long, so we tried to get the ones that we believe are  the most impactful.

-------wr_model.ipynb is the code that we are using to organize data into a usuable format for machine learning and then we are going to use a neural network to train predicting the winrate given a game state. The type of model might change but right now I believe a neural network will be the best since the model will be able to learn about how each parameter affects the winrate given a state of the game. 

It is not done yet as the data is not readily able to be put directly into a ml model. 

We are embedding the pokemon names so that the model is able to learn the roles of the pokemon given their name by itself. 

We also need to embed all possible moves so that we can make a model that predict what move the opponent will choose or if they will switch out their pokemon. This model is needed because when considering the minimax player there are simply too many branches we need to explore each turn. This is where our model that predicts moves comes in. 

If we are able to get a decent model that predicts moves then we can manually prune the amount of branches we need to search, some moves the pokemon might have are improbable and not likely to happen so we can tell our model to ignore those possibilities and focus on the ones that are more likely to happen. 

As for why we need this optimization lets consider the start of the game turn 1, the agent has 9 possible moves. 4 pokemon moves and 5 switches, since turns in pokemon occur at the same time we also need to consider that the opponent has the same amount of moves so 9^9, which is already a lot, but then you realize that each move the pokemon makes has its own random branches it can create, lets take a simple move like rock slide, theres a chance to crit, miss,  cause the enemy to flinch, or none of these so just knowing we will press this move there are 4^9 branches to explore and we have 4 moves, 4*4^9^9 there are a insane number of possible game states just off 1 turn. But some of these are not very probable to occur so we can narrow down our search my removing the moves and switches we dont want to consider.

Another reason we need this model is because we aren't given the possible moves the opponent can do in each game state (just how pokemon works), so in order to simulate the opponent's best move we need to predict their move.

NOTE: right now we are looking at alpha beta pruning, but if we find that this is too ineffcient then we will explore other models, I want to implement it first and then see what happens, also since we know exploring so many branches will slow we want to implement multithreading in the future.

Micah: The code I worked on was part of the max_damage_agent, which Robin made some changes too and the minimax_agent, which I did the majority of. Max Damage is an agent that deciphers the best move to use based on a damage calculation written by Robin that takes into account  multiple factors that allow for damage in Pokemon. Although it could be further explored for a more detailed calculation, that would take a very long time to figure out as there are multiple aspects that would need to be accounted for. It also has a type_matchup function that classifies the damage multiplier in order to determine if a switch would result in probable more damage. The main code I worked on was the Minimax Agent which was a little bit of a nightmare. There is a similar structure to the MaxAgent, however there are a vast number of differences. It has an initialization for the class that sets the depth of the minimax search for later on. It has a choose_move function that essentially calls the minimax function and returns the best move for our particular case; if there is no move or switch available it will default to a random move. Then the actual minimax function itself is a little complex. The base case is when the depth is reached or the battle finishes before that depth is reached, and it returns the eval state function which I’ll get too. The rest of the function essentially swaps between our bot’s turn (maximizing) and the opponent's turn (minimizing). In theory, we will be implementing a neural network that allows us to fully simulate and evaluate the best choice, but for now this is a basic attempt at iterating through each move and using a rudimentary calculation to choose the best evaluation and therefore the best move. So it iterates, simulates the next move, then recursively calls minimax with the simulated battle state, then returns the eval_state at the base_case until the best_move is chosen. Where the bot’s turn aims at maximizing and the opponent’s turn aims at minimizing. The eval_state function right now just calculates the move with the most damage and adds the hp differential in order to set up a base level evaluation to get minimax going. Type_matchup is the same as max damage. The current issue we have with minimax is the simulate_turn function. We are in a bit of a jam with complexity as far as this aspect goes. Robin goes into detail about the ML aspects that he is working on, as far as me, I am figuring out how to freeze and serialize our battle state into a JSON, and then simulate the battle through the Showdown API, then take that game state, and deserialize it back into our function in order to evaluate a move. The issue is arising with what needs to be included in the JSON in order to make it run efficiently as there is an absurd amount of data that is necessary for the game and what can be handled by the ML model that Robin is working on. Other than that the rest of the Class should be good to go and we are going to incorporate the neural network into the evaluation process. 

## Results

--The mini-max player runs right now but does not search more than a depth of 1, since we dont have a way of simulating next game states right now, and since we don't have a way of knowing our opponent's moves. It does a bit better than the max_base-power_player but worse than the max_damage_agent as it is basically picking the move with the most damage everytime.

Although the minimax isn't really functional We have worked on a max_damage_agent (though a better name right now would be a smart_damage_agent). 

We started with this agent to get familiar with the poke-env library and have an agent that our final agent can compare to. 

This max_damage_agent wins 95% of the time against the random agent and 75% of the time against the max_base-power_player, random agent and max_base-power_player are both agents that came in the poke-env library. 

There is a bug that crashes the agent, and we are working on figuring out what the bug is, but for the majority of runs, the bug doesn't happen. 


## How it should work

The mini-max agent runs and as it runs It explores branches by runing the simulation software written in the pokemon showdown repo. It then uses wr_model to predict the winrate/utility of a given state in the game then using that number it uses alpha beta pruning to check what branches it still needs to try exploring.

## What we learnerd

I've learned a lot about what data to consider in then game state of a game and how complicated pokemon is as a competitive game. I also learned a lot about how to think about working on a project from nothing. Going from a blank python file to a whole repository is a neat process and feel a lot less intimidating now. A lot of my time working was figuring out how we were going to put this all together and all the parts we need to complete to get this project done. 1 get comfortable with the poke-env environment, 2 figure out a model to use, 3 figure out what we need to make the model work, 4 start working opn each part, 5 put it together, 6 ..... and there are still more steps to go. Another thing that took up a lot of my time was thinking about how to represent the game state in numbers, such as being able to apply nlp word embedding to the pokemon names and have the model learn the different pokemon as it trains is a fun concept. Parsing all the data was not fun though and gave me a lot of headaches. 

Micah:
As far as what I learned, most of it you can probably see in the above explanation of my code, but I learned that this is a really complex project that I think we underestimated the time that this could take. We are only scratching the surface of what we want to do and develop in order to make an effective agent and model in order to battle competitively in actual Pokemon Showdown battles. I learned a lot about how agents are developed and set up. I made a couple of them and had to really develop a skill of absorbing libraries and being able to utilize them in order to make a functional bot. 